In [1]:
import pandas as pd
import json as json
import ast

In [12]:
# Cargar Data Sets
dfMovies = pd.read_csv('Data/movies_dataset.csv', dtype=str, encoding='UTF-8', decimal='.', quotechar='"')
dfCredits = pd.read_csv('Data/credits.csv', dtype=str, encoding='UTF-8', decimal='.', quotechar='"')

Transformaciones

In [13]:
# Verifica si hay filas duplicadas en dfCredits
duplicados = dfMovies.duplicated()

# Cuenta el número de filas duplicadas
cantidad_duplicados = duplicados.sum()

print(f"Hay {cantidad_duplicados} filas duplicadas")


Hay 17 filas duplicadas


In [14]:
# Verifica si hay filas duplicadas en dfCredits
duplicados = dfCredits.duplicated()

# Cuenta el número de filas duplicadas
cantidad_duplicados = duplicados.sum()

print(f"Hay {cantidad_duplicados} filas duplicadas")

Hay 37 filas duplicadas


In [18]:
# Elimina filas duplicadas y mantiene la primera ocurrencia
dfMovies = dfMovies.drop_duplicates()

dfCredits = dfCredits.drop_duplicates()

In [21]:
# Conviertir las cadenas de texto a listas de diccionarios usando ast.literal_eval
dfCredits['cast'] = dfCredits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
dfCredits['crew'] = dfCredits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [25]:
# Renombrar Columna
dfCredits.rename(columns={'id': 'id_pelicula'}, inplace=True)

In [23]:
def expand_and_normalize(df, column):
    """
    Expande una columna de listas en filas y normaliza los diccionarios dentro de esa columna.

    Parámetros:
    df (pd.DataFrame): DataFrame que contiene la columna a procesar.
    column (str): Nombre de la columna que contiene listas de diccionarios para expandir y normalizar.

    Retorna:
    pd.DataFrame: Un nuevo DataFrame con la columna original expandida y normalizada.
    """
    
    # Verificar si la columna especificada existe en el DataFrame
    if column not in df.columns:
        raise KeyError(f"La columna '{column}' no se encuentra en el DataFrame.")
    
    # Explota la columna de listas en filas individuales
    # Cada elemento de la lista en una fila se convierte en una fila separada
    df_exploded = df.explode(column)
    
    # Normaliza los diccionarios de la columna explotada
    # Esta operación convierte cada diccionario en una fila de columnas separadas
    df_normalized = pd.json_normalize(df_exploded[column])
    
    # Añade las columnas del DataFrame normalizado al DataFrame explotado
    for col in df_normalized.columns:
        # Crear un nuevo nombre para cada columna normalizada, usando el nombre original de la columna
        new_col_name = f"{col}_{column}"
        # Añadir la columna al DataFrame explotado
        df_exploded[new_col_name] = df_normalized[col].values
    
    # Eliminar la columna original que fue normalizada, ya que ahora está representada por las nuevas columnas
    df_exploded = df_exploded.drop(columns=[column])
    
    # Retorna el DataFrame con la columna original explotada y las nuevas columnas normalizadas
    return df_exploded


In [26]:
# Desanida columnas del data frame Credits
dfCreditsNormal = expand_and_normalize(dfCredits, 'cast')
dfCreditsNormal = expand_and_normalize(dfCreditsNormal, 'crew')

In [29]:
# Convierte las cadenas de texto a listas de diccionarios usando ast.literal_eval
dfMovies['belongs_to_collection'] = dfMovies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

C:\Users\Pablo\AppData\Local\Temp\ipykernel_7820\2657334609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfMovies['belongs_to_collection'] = dfMovies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [35]:

# Rellena con 0 los campos de las columnas:
dfMovies[['revenue', 'budget']] = dfMovies[['revenue', 'budget']].fillna(0)
dfMovies = dfMovies.dropna(subset=['release_date'])

# Eliminar filas donde 'release_date' es nulo
dfMovies['release_date'] = pd.to_datetime(dfMovies['release_date'], errors='coerce')

# Crear la columna release_year usando .loc para evitar el SettingWithCopyWarning
dfMovies.loc[:, 'release_year'] = dfMovies['release_date'].dt.year

# Convertir en datos numericos las columnas:
dfMovies['budget'] = pd.to_numeric(dfMovies['budget'], errors='coerce')
dfMovies['revenue'] = pd.to_numeric(dfMovies['revenue'], errors='coerce')
dfMovies['runtime'] = pd.to_numeric(dfMovies['runtime'], errors='coerce')
dfMovies['vote_average'] = pd.to_numeric(dfMovies['vote_average'], errors='coerce')
dfMovies['vote_count'] = pd.to_numeric(dfMovies['vote_count'], errors='coerce')

# Crean columna return
dfMovies['return'] = dfMovies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Eliminnar columnas:
columns = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
dfMovies.drop(columns=columns, inplace=True)

dfMovies.rename(columns={'id': 'id_pelicula'}, inplace=True)

KeyError: "['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'] not found in axis"

In [38]:
# Desanida columnas del data frame Movies
dfMoviesNormal = expand_and_normalize(dfMovies, 'belongs_to_collection')

In [39]:
# Convierte las cadenas de texto a listas de diccionarios usando ast.literal_eval
dfMoviesNormal['genres'] = dfMoviesNormal['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
dfMoviesNormal['production_companies'] = dfMoviesNormal['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
dfMoviesNormal['production_countries'] = dfMoviesNormal['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
dfMoviesNormal['spoken_languages'] = dfMoviesNormal['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [40]:
# Desanida columnas del data frame Movies
dfMoviesNormal = expand_and_normalize(dfMoviesNormal, 'genres')
dfMoviesNormal = expand_and_normalize(dfMoviesNormal, 'production_companies')
dfMoviesNormal = expand_and_normalize(dfMoviesNormal, 'production_countries')
dfMoviesNormal = expand_and_normalize(dfMoviesNormal, 'spoken_languages')


* Exportar a CVS

In [ ]:
dfMoviesNormal.to_csv('dfMoviesNormal.csv', index=False)
dfCreditsNormal.to_csv('dfCreditsNormal.csv', index=False)